BDLE 2022

date du document  :  22/09/2022

# TP1 Préparation de données


Restructuration de données et analyse de données.


## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [1]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [2]:
!pip install -q pyspark
!pip install -q findspark

Démarrer la session spark

In [3]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [4]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1665956064618


In [5]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [6]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [7]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [8]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    for r in tabRequetes:
        if len(r.strip()) > 2:
          derniere = spark.sql(r)
    return display(derniere)

In [9]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [10]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [11]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['userVisits-Toro.csv', 'POI-Toro.csv']

In [12]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

userVisits-Toro.csv is already stored
POI-Toro.csv is already stored


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [13]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [14]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [15]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits


,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [16]:
%%sql
select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

**POI_sequence**(seqID, userID, poiID). La liste des POI d'une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

In [17]:
%%sql
create or replace temp view POI_sequence as 
select distinct seqID, userID, poiID
from user_visits;

select * from POI_sequence

,seqID,userID,poiID
0,3,10012675@N05,6
1,4,10012675@N05,13
2,9,10014440@N06,24
3,14,10014440@N06,24
4,46,10316646@N07,3
...,...,...,...
95,619,18412989@N00,22
96,619,18412989@N00,16
97,620,18412989@N00,22
98,622,18412989@N00,16


### Les POI

In [18]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [19]:
%%sql
cache table POI;

SELECT * 
From POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [20]:
%%sql
create or replace temp view themes as
select distinct theme
from POI;

cache table themes;

select * from themes

,theme
0,Beach
1,Amusement
2,Shopping
3,Cultural
4,Structure
5,Sport


## Exercice 1

#### 1) Les 10 POI les plus visités

In [21]:
%%sql 

select p.poiName
from user_visits u, POI p
where u.poiID = p.poiID 
group by u.poiID, p.poiName
order by count(u.userID) desc
limit 10;


,poiName
0,Royal_Ontario_Museum
1,Nathan_Phillips_Square
2,Yonge-Dundas_Square
3,CN_Tower
4,Air_Canada_Centre
5,Rogers_Centre
6,Hockey_Hall_of_Fame
7,Toronto_Eaton_Centre
8,Ripley%27s_Aquarium_of_Canada
9,Kensington_Market


#### 2) POI avec date
POI_date(...) Les POI avec la date détaillée composée des attributs : année, mois, jour, heure.

Completer les visites avec une date detaillée

In [22]:
%%sql

create or replace temp view user_visits2 as

select photoID, userID, TIMESTAMP(date) , poiID , poiTheme, poiFreq , seqID
from user_visits; 

cache table user_visits2;

select * from user_visits2



,photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,7941504100,10007579@N00,2012-09-05 11:31:28,30,Structure,1538,1
1,4886005532,10012675@N05,2006-03-19 01:30:48,6,Cultural,986,2
2,4886006468,10012675@N05,2006-03-19 01:37:28,6,Cultural,986,2
3,4885404441,10012675@N05,2006-03-19 01:39:33,6,Cultural,986,2
4,4886008334,10012675@N05,2006-03-19 01:40:45,6,Cultural,986,2
...,...,...,...,...,...,...,...
95,2654929774,10014440@N06,2008-07-09 08:53:33,25,Shopping,1701,10
96,2654104231,10014440@N06,2008-07-09 08:53:54,25,Shopping,1701,10
97,2654930912,10014440@N06,2008-07-09 08:54:10,25,Shopping,1701,10
98,2654105249,10014440@N06,2008-07-09 08:54:15,25,Shopping,1701,10


#### 3) Le nombre de POI par utilisateur

In [23]:
%%sql 

select userID , count(*)
from user_visits2
group by userID;


,userID,count(1)
0,11090433@N05,71
1,12641532@N00,8
2,12693966@N07,211
3,12874594@N08,10
4,13411198@N00,5
...,...,...
95,60261913@N00,2
96,60365624@N06,1
97,61117412@N07,2
98,62077150@N06,16


#### 4) Trajectoire
Trajectoire(userID, seqID, listPOI)
Pour chaque séquence, la liste ORDONNEE des POI visités

Liste ordonnée par date des poiID visites

In [24]:
# fonction qui regroupe les doublons successif

def myfunc (liste) :
  new = list()
  i = 0
  while i < (len(liste)): 
    j = i
    new.append(liste[i])
    while j < len(liste)  and  liste[i] == liste[j]:
      j = j + 1
    i = j
  return new

#myfunc([1,1,2,3,4,4,1,1])

In [25]:
spark.udf.register("myfunc", myfunc)

<function __main__.myfunc(liste)>

In [26]:
%%sql 

create or replace temp view user_visits3 as


select seqID,userID, myfunc(collect_list(cast(poiID as int))) as liste_poi
from (select * from user_visits2  order by date)
group by  seqID ,userID
;


cache table user_visits3;

select * from user_visits3;


,seqID,userID,liste_poi
0,6,10014440@N06,[23]
1,17,10031363@N00,[7]
2,21,10031363@N00,[3]
3,42,10316646@N07,[6]
4,44,10316646@N07,[6]
...,...,...,...
95,657,19511310@N00,[22]
96,659,19611789@N00,"[7, 1]"
97,669,19761391@N06,[23]
98,670,19761391@N06,[13]


#### 5) Transitions
Transitions(poi1, poi2, transitions) avec nb étant le nombre de déplacements directs de poi1 à poi2

In [27]:
def myfunc2 (poi, liste) :
  l = []
  liste = list(liste)
  for i in range(len(poi)):
    count = 0
    for j in range(len(poi)-1):
      for lis in liste:
        for i in range(len(lis)-1):
          if lis[i] == poi[i]:
            if lis[i+1] == poi[j]:
              count += 1
    l.append([poi[i],poi[j], count])
  return l

spark.udf.register("myfunc2", myfunc2)
#myfunc2(1,2, [[1,2,3], [5,6,7,8], [6,7]])

<function __main__.myfunc2(poi, liste)>

In [ ]:
%%sql 

select myfunc2(collect_list(v.poiID),collect_set(w.liste_poi)) as trans
from  user_visits3 w , POI v
limit 200

-- utilisé deux tables, passer en parametre les deux pois et la liste des pois

#### 6) Distances entre POIs
Distance(poi1, poi2, distance)

In [ ]:
from math import sin, cos, sqrt, atan2

def distance(lat1, lat2, lon1, lon2):
  R = 6373.0
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  distance = R * c
  return distance
  
spark.udf.register("distance", distance)

In [ ]:
%%sql 

select p1.poiID as poi1 , p2.poiID as poi2 , distance(p1.latitude, p2.latitude, p1.longitude, p2.longitude) as distance
from POI p1, POI p2
where p1.poiID <> p2.poiID


#### 7) DuréeVisitePOI
La durée moyenne de visite d'un POI.

In [ ]:
%%sql

select poiID , avg(duree) as duree
from (select poiID , last(date)-first(date) as duree from user_visits2  group by seqID, userID, poiID)
group by poiID

In [ ]:
%%sql 
select poiID , last(date)-first(date) from user_visits2  group by seqID, userID, poiID

## Exercice 2

Charger les données de Geonames

In [ ]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.zip"
with zipfile.ZipFile(local_file, 'r') as zip_ref:
  zip_ref.extractall(local_dir)

os.listdir(local_dir)

In [ ]:
os.listdir(local_dir)

In [ ]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")

geonames.show(3)


#### 1) Geonames
Définir le schéma, limité au 9 premiers attributs, d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [ ]:
schema = "geonameID long, name String, asciiName String, alternateNames String, latitude double, longitude double, feature_class String,feature_code String,country_code String "

geoname = spark.read.option("header", "True").option("delimiter",  "\t").csv(local_dir + "/" + "allCountries.txt", schema = schema)
geoname.persist()
geoname.createOrReplaceTempView("geoname")
geoname.show(10)

#### 2) Association entre les POI et Geonames
Compléter les POI avec des attributs de geonames (par exemple name et feature code)

In [ ]:
%%sql 

create or replace temp view POI2 as
select p.poiID,	p.poiName,	p.latitude,	p.longitude, p.theme, g.name,g.feature_class, g.feature_code , g.country_code ,g.geonameID
from geoname g, POI p
where g.latitude == p.latitude and g.longitude == p.longitude


In [ ]:
%%sql 
cache table POI2;
select * from POI2 limit 5;